# 4. Python Syntax — Expanded (EN)

**Goals**
- Master core Python syntax and patterns used in real projects.
- Learn variable rules & scope, control flow, functions, exceptions, I/O, f‑strings, comprehensions, iterators/generators, decorators, dataclasses, enums, context managers, and modules.
- Practice with short exercises. (Look for `# TODO` cells.)

## 1) Variables, Names, and Scope

- Names bind to *objects* (everything is an object).
- Scope levels: **LEGB** (Local, Enclosing, Global, Builtins).
- Rebinding does not copy for mutable types; be careful with shared references.

In [23]:
x = 10                 # int object bound to name x
y = x                  # y references same int object (ints are immutable)
x = 20                 # x now bound to a different int; y unchanged
print("x:", x, "y:", y)

# Mutable example
a = [1,2,3]
b = a                  # both names point to the same list
b.append(99)
print("a:", a, "b:", b)  # both changed because they share the same list

x: 20 y: 10
a: [1, 2, 3, 99] b: [1, 2, 3, 99]


In [24]:
# LEGB & 'global'/'nonlocal' demo
message = "global"

def outer():
    message = "enclosing"
    def inner():
        nonlocal message    # refers to 'enclosing' scope
        message = "changed in inner"
        return message
    inner()
    return message

print("outer() ->", outer())
print("global message is still:", message)

outer() -> changed in inner
global message is still: global


## 2) Control Flow Patterns

- `if/elif/else`, conditional expressions.
- `for` with `enumerate`, `zip`, and `for ... else`.
- `while` loops with `break/continue`.
- `match/case` (Python 3.10+) for simple structural matching.

In [25]:
# Conditional expression (ternary)
score = 83
label = "pass" if score >= 60 else "fail"
print("score:", score, "->", label)

# for + enumerate + zip
names = ["Ada","Grace","Linus"]
points = [10, 15, 12]
for i, (n, p) in enumerate(zip(names, points), start=1):
    print(f"{i}. {n} -> {p} points")

# for ... else: 'else' runs if the loop wasn't broken
nums = [2,3,5,7,9,11]
target = 9
for v in nums:
    if v == target:
        print("found", v)
        break
else:
    print("not found")  # won't run because we broke

score: 83 -> pass
1. Ada -> 10 points
2. Grace -> 15 points
3. Linus -> 12 points
found 9


In [26]:
# match/case (requires Python 3.10+)
def http_status(code: int) -> str:
    match code:
        case 200 | 201:
            return "OK"
        case 400:
            return "Bad Request"
        case 404:
            return "Not Found"
        case 500:
            return "Server Error"
        case _:
            return "Unknown"

print(http_status(404), http_status(201), http_status(123))

Not Found OK Unknown


## 3) Comprehensions

- List/Dict/Set comprehensions are concise and fast.
- Use conditional filters inside comprehensions.

In [27]:
squares = [x*x for x in range(6)]
even_squares = [x for x in squares if x % 2 == 0]
index_map = {i: v for i, v in enumerate("PYTHON")}
unique_letters = {c.lower() for c in "Mississippi"}
print(squares, even_squares)
print(index_map)
print(unique_letters)

[0, 1, 4, 9, 16, 25] [0, 4, 16]
{0: 'P', 1: 'Y', 2: 'T', 3: 'H', 4: 'O', 5: 'N'}
{'m', 'p', 's', 'i'}


## 4) Functions: Arguments, Defaults, *args/**kwargs, Closures

- Default values are evaluated once (avoid mutable defaults).
- Use `*args` for positional varargs, `**kwargs` for keywords.
- Closures capture variables from the enclosing scope.

In [28]:
def append_item(value, lst=None):
    # Avoid mutable default like lst=[]
    if lst is None:
        lst = []
    lst.append(value)
    return lst

print(append_item(1), append_item(2), append_item(3, ['existing']))

[1] [2] ['existing', 3]


In [29]:
def mix(a, b=10, *args, sep="-", **kw):
    parts = [f"a={a}", f"b={b}"]
    parts.extend(map(str, args))
    parts.extend(f"{k}={v}" for k, v in kw.items())
    return sep.join(parts)

print(mix(1, 2, 3, 4, sep="|", mode="demo"))

a=1|b=2|3|4|mode=demo


In [30]:
# Closures and 'nonlocal'
def make_counter():
    count = 0
    def inc(step=1):
        nonlocal count
        count += step
        return count
    return inc

c = make_counter()
print(c(), c(), c(5))

1 2 7


## 5) Exceptions & Custom Errors

- Use `try/except/else/finally` to handle errors.
- Raise specific exceptions; create custom ones when helpful.

In [31]:
class DataFormatError(Exception):
    """Custom exception for invalid data format."""

def parse_int(s: str) -> int:
    try:
        return int(s)
    except ValueError as e:
        raise DataFormatError(f"Not an integer: {s}") from e

for s in ["42", "x"]:
    try:
        print("parse_int:", s, "->", parse_int(s))
    except DataFormatError as e:
        print("Handled:", e)

parse_int: 42 -> 42
Handled: Not an integer: x


## 6) Files & Context Managers (`with`)

- `with` guarantees cleanup (close files).
- Build your own context manager with `contextlib`.

In [32]:
from contextlib import contextmanager

@contextmanager
def tagged(name):
    print(f"<start {name}>")
    try:
        yield
    finally:
        print(f"</end {name}>")

with tagged("demo"), open("demo.txt", "w", encoding="utf-8") as f:
    f.write("Hello!\n")

print(open("demo.txt", encoding="utf-8").read().strip())

<start demo>
</end demo>
Hello!


## 7) f-Strings & Formatting

- Format specifiers: width, precision, alignment, thousands separator, datetime.

In [33]:
from datetime import datetime
pi = 3.1415926535
n = 1234567
when = datetime(2025, 8, 21, 10, 0, 0)

print(f"pi ~ {pi:.3f}")           # precision
print(f"{n:,}")                  # thousands sep
print(f"|{pi:>10.2f}|")          # right align in width 10
print(f"{when:%Y-%m-%d %H:%M}")  # datetime formatting

pi ~ 3.142
1,234,567
|      3.14|
2025-08-21 10:00


## 8) Iterators & Generators

- Any object with `__iter__` is iterable.
- Generators (`yield`) produce values lazily.
- Generator expressions are memory-efficient.

In [34]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1

print(list(countdown(5)))

# Generator expression
squares = (i*i for i in range(5))
print(next(squares), next(squares))
print(sum(squares))  # uses remaining values

[5, 4, 3, 2, 1]
0 1
29


## 9) Decorators (functions that wrap functions)

- Decorators add behavior (logging, caching, auth) without changing the wrapped function.

In [35]:
import time
from functools import wraps

def timing(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        t0 = time.perf_counter()
        try:
            return fn(*args, **kwargs)
        finally:
            dt = time.perf_counter() - t0
            print(f"{fn.__name__} took {dt*1000:.2f} ms")
    return wrapper

@timing
def slow_add(a, b):
    time.sleep(0.05)
    return a + b

print("slow_add:", slow_add(2, 3))

slow_add took 55.03 ms
slow_add: 5


## 10) Dataclasses & Enum

- `dataclasses` generate boilerplate (`__init__`, `__repr__`, comparisons).
- `Enum` provides named constants.

In [36]:
from dataclasses import dataclass
from enum import Enum, auto

class Role(Enum):
    USER = auto()
    ADMIN = auto()

@dataclass
class User:
    name: str
    role: Role = Role.USER
    active: bool = True

u = User("Alice", role=Role.ADMIN)
print(u)
print(u.role is Role.ADMIN)

User(name='Alice', role=<Role.ADMIN: 2>, active=True)
True


## 11) Modules & Imports

- `import module`, `from module import name`, or `as alias`.
- `if __name__ == "__main__":` for runnable modules.

In [37]:
# Install package
!pip install geom

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.12/site-packages/pypls-1.0.3-py3.12-macosx-15.0-arm64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [38]:
import sys, importlib, os, pathlib

# 1) Write the module file to the *current working directory*
mod_code = """
def area_circle(r):
    from math import pi
    return pi * r * r

if __name__ == '__main__':
    print('area of r=2 ->', area_circle(2))
"""
path = pathlib.Path("geom.py").resolve()
path.write_text(mod_code, encoding="utf-8")

# 2) Make sure the directory is on sys.path
cwd = str(path.parent)
if cwd not in sys.path:
    sys.path.insert(0, cwd)

# 3) Invalidate import caches and remove any old module object
importlib.invalidate_caches()
sys.modules.pop("geom", None)

# 4) Import fresh and verify where it came from
geom = importlib.import_module("geom")
print("Imported geom from:", getattr(geom, "__file__", "<no __file__>"))
print("Exported names:", [n for n in dir(geom) if not n.startswith("_")])

# 5) Use it
print("area r=3:", geom.area_circle(3))

Imported geom from: /Volumes/Extreme SSD/Github/Python-101/App/notebooks/geom.py
Exported names: ['area_circle']
area r=3: 28.274333882308138


## 12) Exercises (Short)

1. **FizzBuzz**: Print numbers 1..50. For multiples of 3 print `Fizz`, of 5 print `Buzz`, of both print `FizzBuzz`.
2. **Unique Words**: Given a sentence, create a dictionary mapping each lowercase word to its count.
3. **Anonymize CSV**: Read a CSV file with a column `email` and output a list of masked emails (keep only the domain).
4. **Temperature Converter**: Write a function that accepts `value` and `unit` (`'C'` or `'F'`) and converts to the other.
5. **Decorator Practice**: Write a decorator that retries a function up to 3 times if it raises a `ValueError`.

In [39]:
# TODO 1) FizzBuzz
for i in range(1, 51):
    out = ""
    if i % 3 == 0: out += "Fizz"
    if i % 5 == 0: out += "Buzz"
    print(out or i)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz


In [40]:
# TODO 2) Unique Words
text = "To be or not to be, that is the Question."
counts = {}
for token in text.lower().replace(",", "").replace(".", "").split():
    counts[token] = counts.get(token, 0) + 1
counts

{'to': 2,
 'be': 2,
 'or': 1,
 'not': 1,
 'that': 1,
 'is': 1,
 'the': 1,
 'question': 1}

In [41]:
# TODO 3) Anonymize CSV (demo with in-memory rows)
rows = [
    {"email":"alice@example.com"},
    {"email":"bob@lab.org"},
    {"email":"carol@uni.ac.th"}
]
masked = [r["email"].split("@")[-1] for r in rows]
masked

['example.com', 'lab.org', 'uni.ac.th']

In [42]:
# TODO 4) Temperature Converter
def convert_temp(value: float, unit: str) -> float:
    if unit.upper() == "C":
        return value * 9/5 + 32
    elif unit.upper() == "F":
        return (value - 32) * 5/9
    else:
        raise ValueError("unit must be 'C' or 'F'")

print(convert_temp(100, "C"), convert_temp(32, "F"))

212.0 0.0


In [43]:
# TODO 5) Retry Decorator
import random, time
from functools import wraps

def retry3(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        last = None
        for _ in range(3):
            try:
                return fn(*args, **kwargs)
            except ValueError as e:
                last = e
                time.sleep(0.01)
        raise last
    return wrapper

@retry3
def flaky():
    if random.random() < 0.6:
        raise ValueError("boom")
    return "ok"

# Try a few times
for _ in range(5):
    try:
        print(flaky())
    except ValueError as e:
        print("failed:", e)

ok
ok
ok
failed: boom
ok
